In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
model = Network('Model')
# Block 1: nodes
clientDelay = Delay(model, 'Client')
cacheNode = Cache(model, 'Cache', 1000, 50, ReplacementStrategy.LRU)
cacheDelay = Delay(model, 'CacheDelay')

In [3]:
# Block 2: classes
clientClass = ClosedClass(model, 'ClientClass', 1, clientDelay, 0)
hitClass = ClosedClass(model, 'HitClass', 0, clientDelay, 0)
missClass = ClosedClass(model, 'MissClass', 0, clientDelay, 0)

clientDelay.set_service(clientClass, Immediate())
cacheDelay.set_service(hitClass, Exp.fitMean(0.2))
cacheDelay.set_service(missClass, Exp.fitMean(1.0))

cacheNode.set_read(clientClass, Zipf(1.4, 1000))
cacheNode.set_hit_class(clientClass, hitClass)
cacheNode.set_miss_class(clientClass, missClass)

In [4]:
# Block 3: topology
P = model.init_routing_matrix()
# routing from client to cache
P.set(clientClass, clientClass, clientDelay, cacheNode, 1.0)
# routing out of the cache
P.set(hitClass, hitClass, cacheNode, cacheDelay, 1.0)
P.set(missClass, missClass, cacheNode, cacheDelay, 1.0)
# return to the client
P.set(hitClass, clientClass, cacheDelay, clientDelay, 1.0)
P.set(missClass, clientClass, cacheDelay, clientDelay, 1.0)
# routing from cacheNode
model.link(P)

In [5]:
ssaAvgTablePara = SolverSSA(model, samples=20000, seed=1, verbose=True, method='serial').avg_table()

SSA samples:   20000
SSA analysis [method: serial, lang: java, env: 21.0.8] completed in 12.018969s. Iterations: 20001.
      Station     JobClass    QLen    Util       RespT  ResidT  ArvR    Tput
0      Client  ClientClass  0.0000  0.0000  1.0000e-08  0.0000   0.0  3.0155
4  CacheDelay     HitClass  0.4814  0.4814  2.0000e-01  0.1640   0.0  2.4068
5  CacheDelay    MissClass  0.5186  0.5186  1.0000e+00  0.1798   0.0  0.5186
